# Chapter 17 - Reinforcement Learning

Reinforcement Learning (RL) is a suite of techniques to build machine learning systems that take decisions sequentially.

The key distinction between reinforcement learning and standard deep learning is that in standard deep learning the prediction of a trained model on one test datum does not affect the predictions on a future test datum; in reinforcement learning decisions at future instants (in RL, decisions are also called actions) are affected by what decisions were made in the past.

## 17.1. Markov Decision Process (MDP)

### 17.1.1. Definition of an MDP

A Markov decision process (MDP) is a model for how the state of a system evolves as different actions are applied to the system. An MDP is defined by the following components:
* The set of states $\mathcal{S}$ in the MDP.
* The set of actions $\mathcal{A}$ that an agent can take. Actions can change the current state of the agent to some other state within the set $\mathcal{S}$.
* We may not know how the agent's state changes exactly but only know it up to approximation. In this case, there is a transition probability function $T: \mathcal{S} \times \mathcal{A} \times \mathcal{S} \rightarrow [0, 1]$ such that $T(s, a, s') = P(s' \mid s, a)$ using the conditional probability of eaching a state $s'$ given that the robot was at state $s$ and took an action $a$. The transition function is a probability distribution and we therefore have $\sum_{s' \in \mathcal{S}} T(s, a, s') = 1$ for all $s \in \mathcal{S}$ and $a \in \mathcal{A}$.
* A reward function $R: \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R}$ that gives a reward to the agent for taking an action $a$ at state $s$. If the reward $r(s,a)$ is large, this indicates that taking the action $a$ at state $s$ is more useful to achieving the goal of the agent. The reward is designed by the user with the goal in mind.

### 17.1.2. Return and Discount Factor

A Markov decision process (MDP) is defined as
\begin{split}
\textrm{MDP}: (\mathcal{S}, \mathcal{A}, T, r).
\end{split}

When an agent starts at a particular state $s_0 \in \mathcal{S}$ and continues taking actions, the agent will end up in a trajectory
\begin{split}
\tau = (s_0, a_0, r_0, s_1, a_1, r_1, s_2, a_2, r_2, \ldots).
\end{split}

At each time step $t$, the agent is at a state $s_t$ and takes an action $a_t$, which results in a reward $r_t=r(s_t, a_t)$. The *return* of a trajectory is the total reward obtained by the agent along such a trajectory:
\begin{split}
R(\tau) = r_0 + r_1 + r_2 + \cdots.
\end{split}

The objective in reinforcement learning is to find a trajectory that has the largest *return*.

The sequence of states and actions in a trajectory can be infinitely long and the return of any such infinitely long trajectory will be infinite. In order to keep the reinforcement learning formulation meaningful even for such trajectories, we introduce the notion of a discount facotr $\gamma \in [0, 1)$ and define the *discounted return* as
\begin{split}
R(\tau) = r_0 + \gamma r_1 + \gamma^2 r_2 + \cdots = \sum_{t=0}^\infty \gamma^t r_t.
\end{split}

If $\gamma$ is very small, the rewards earned by the agent in the far future, say $t=1000$, are heavily discounted by the factor $\gamma^{1000}$. This encourages the agent to select short trajectories that achieve its goal.

For large values of the discount factor, say $\gamma=0.99$, the agent is encouraged to *explore* and then find the best trajectory to go to the goal state.

### 17.1.3. Discussion of the Markov Assumption

Markov systems are all systems where the next state $s_{t+1}$ is only a function of the current state $s_t$ and the action $a_t$ taken at the current state. In Markov systems, the next state does not depend on which actions were taken in the past or the states that the robot was at in the past.

## 17.2. Value Iteration

### 17.2.1. Stochastic Policy

A *stochastic policy* denoted as $\pi(a \mid s)$ (policy for short) is a conditional distribution over the actions $a \in \mathcal{A}$ given the state $s \in \mathcal{S}$, $\pi(a \mid s) \equiv P(a \mid s)$.

For example, if the agent has 4 actions $\mathcal{A} = \{ \textrm{up}, \textrm{down}, \textrm{left}, \textrm{right} \}$, then the policy at a state $s\in \mathcal{S}$ for such a set of actions $\mathcal{A}$ is a categorical distribution where the probabilities of the 4 actions could be $[0.3, 0.1, 0.4, 0.2]$; at some other state $s' \in \mathcal{S}$, the probabilities $\pi(a \mid s')$ could be $[0.6, 0.1, 0.1, 0.2]$.

Note that $\sum_a \pi(a \mid s) = 1$ for any state $s$.

A deterministic policy is a special case of a stochastic policy in that the distribution $\pi(a \mid s)$ only gives non-zero probability to one particular action, e.g., $[1,0,0,0]$ with 4 actions.

We will use $\pi(s)$ to denote the conditional distribution over the actions $a \in \mathcal{A}$ given the state $s \in \mathcal{S}$, $\pi(s) \equiv \pi(a \mid s)$.

### 17.2.2. Value Function

Suppose that the agent starts at a state $s_0$, and at each time step, it first samples an action from the policy $a_t \sim \pi(s_t)$ and takes this action to result in the next state $s_{t+1}$. 

The trajectory $\tau = (s_0, a_0, r_0, s_1, a_1, r_1, \ldots)$ can be different dpending upon which particular action $a_t$ is sampled at intermediate time steps.

The average *return* $R(\tau) = \sum_{t=0}^\infty \gamma^t r(s_t, a_t)$ of all such trajectories is defined as
\begin{split}
V^\pi(s_0) = E_{a_t \sim \pi(s_t)} \Big[ R(\tau) \Big] = E_{a_t \sim \pi(s_t)} \Big[ \sum_{t=0}^\infty \gamma^t r(s_t, a_t) \Big],
\end{split}
where $s_{t+1} \sim P(s_{t+1} \mid s_t, a_t)$ is the next state of the agent and $r(s_t, a_t)$ is the instantaneous reward obtained by taking action $a_t$ in state $s_t$ at time $t$. This is called the *value function* for the policy $\pi$.

In other words, the value of a state $s_0$ for a policy $\pi$, denoted by $V^\pi(s_0)$, is the expected $\gamma$-discounted *return* obtained by the agent if ti begins at state $s_0$ and take actions from the policy $\pi$ at each time step.

The trajectory $\tau$ can be broken down into the following components:
1. the first stage which corresponds to $s_0 \to s_1$ upon taking the action $a_0$, and
2. a second stage which is the trajectory $\tau' = (s_1, a_1, r_1, \ldots)$ thereafter.

The key idea behind all algorithms in reinforcement learning is that *the value of state $s_0$ can be written as the average reward obtained in the first stage and the value function averaged over all possible next stages $s_1$.

This arises from the Markov assumption: *the average return from the current state is the sum of the average return from the next state and the average reward of going to the next state*. Mathematically, we can rewrite the two stages as
\begin{split}
V^\pi(s_0) = r(s_0, a_0) + \gamma\ E_{a_0 \sim \pi(s_0)} \Big[ E_{s_1 \sim P(s_1 \mid s_0, a_0)} \Big[ V^\pi(s_1) \Big] \Big].
\end{split}

This is the foundation of the principle of *dynamic programming* upon which all reinforcement learning algorithms are based. The second stage gets two expectations, one over the choices of the action $a_0$ taken in the first stage using the sthchastic policy, and the other over the possible states $s_1$ obtained from the chosen action.

Using the transition probability in the Markov decision process, we can rewrite the above equation as
\begin{split}
V^\pi(s) = \sum_{a \in \mathcal{A}} \pi(a \mid s) \Big[ r(s,  a) + \gamma\  \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V^\pi(s') \Big];\ \textrm{for all } s \in \mathcal{S}.
\end{split}

### 17.2.3. Action-Value Function

In practical implementations, we often maintain a quantity called the "action value" function which is closely related quantity to the value function. The action-value function $Q^\pi(s_0, a_0)$ is defined to be the average *return* of a trajectory that begins at state $s_0$ but when the action of the first stage is fixed to be $a_0$:
\begin{split}
Q^\pi(s_0, a_0) = r(s_0, a_0) + E_{a_t \sim \pi(s_t)} \Big[ \sum_{t=1}^\infty \gamma^t r(s_t, a_t) \Big],
\end{split}
where the summation inside the expectation is from $t=1,\ldots,\infty$ because the reward of the first stage is fixed in this case. We can also break down the trajectory into two parts and write
\begin{split}
Q^\pi(s, a) = r(s, a) + \gamma \sum_{s' \in \mathcal{S}} P(s' \mid s, a) \sum_{a' \in \mathcal{A}} \pi(a' \mid s')\ Q^\pi(s', a');\ \textrm{ for all } s \in \mathcal{S}, a \in \mathcal{A}.
\end{split}
This version for the action value function is the analog of
\begin{split}
V^\pi(s) = \sum_{a \in \mathcal{A}} \pi(a \mid s) \Big[ r(s,  a) + \gamma\  \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V^\pi(s') \Big];\ \textrm{for all } s \in \mathcal{S}.
\end{split}

### 17.2.4. Optimal Stochastic Policy

Both the value function and the action-value function depend upon the policy that the agent chooses. The *optimal policy* is the policy that achieves the maximal average return:
\begin{split}
\pi^* = \underset{\pi}{\mathrm{argmax}} V^\pi(s_0).
\end{split}

Of all possible stochastic policies that the agent could have taken, the optimal policy $\pi^*$ achieves the largest average discounted *return* for trajectories starting from the initial state $s_0$. The value function and the action-value function for the optimal policy are denoted by $V^* \equiv V^{\pi^*}$ and $Q^* \equiv Q^{\pi^*}$, respectively.

For a deterministic policy where there is only one action that is possible under the policy at any given state, the optimal policy is also deterministic. In this case, the optimal policy is given as
\begin{split}
\pi^*(s) = \underset{a \in \mathcal{A}}{\mathrm{argmax}} \Big[ r(s, a) + \gamma \sum_{s' \in \mathcal{S}} P(s' \mid s, a)\ V^*(s') \Big].
\end{split}

The optimal action at state $s$ (for a deterministic policy) is the one that maximizes the sum of reward $r(s,a)$ from the first stage and the average return of the trajectories starting from the next state $s'$, averaged over all possible next states $s'$ from the second stage.

### 17.2.5. Principle of Dynamic Programming

From all the previous sections, we can summarize an algorithm to compute the optimal value function $V^*$ and the action-value function $Q^*$:
\begin{split}
V^*(s) = \sum_{a \in \mathcal{A}} \pi^*(a \mid s) \Big[ r(s,  a) + \gamma\  \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V^*(s') \Big];\ \textrm{for all } s \in \mathcal{S}.
\end{split}

For a deterministic optimal policy $\pi^*$, since ther eis only one action that can be taken at state $s$, 
\begin{split}
V^*(s) = \mathrm{argmax}_{a \in \mathcal{A}} \Big\{ r(s,a) + \gamma \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V^*(s') \Big\}
\end{split}
for all states $s \in \mathcal{S}$. This identity is called the “principle of dynamic programming”.

### 17.2.6. Value Iteration